<a href="https://colab.research.google.com/github/sugarforever/AutoGen-Tutorials/blob/main/autogen_with_function_calls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoGen with Function Calls

Let's see how to use AssistantAgent and UserProxyAgent to make function calls with the new feature of OpenAI models.

In this notebook, we will instruct the AssistantAgent to use provided function as tool to search news with Google Serp API.

In [ ]:
%pip install pyautogen~=0.1.0 docker google-search-results -q -U

In [ ]:
from serpapi import GoogleSearch

def search_google_news(keyword):
  print(f"Keyword: {keyword}")
  search = GoogleSearch({
      "q": keyword,
      "tbm": "nws",
      "api_key": open("serp_api_key.txt").read().strip()
    })
  result = search.get_dict()
  return [item['link'] for item in result['news_results']]


## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.


In [ ]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4"],
    },
)

llm_config={
    "request_timeout": 600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
    "functions": [
        {
            "name": "search_google_news",
            "description": "Search google news by keyword",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "The keyword that's used to search google news",
                    }
                },
                "required": ["keyword"],
            },
        }
    ],
}

## Construct Agents

We construct the assistant agent and the user proxy agent. We specify `human_input_mode` as "TERMINATE" in the user proxy agent, which will ask for human feedback when it receives a "TERMINATE" signal from the assistant agent.

In [ ]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "."},
    llm_config=llm_config,
    system_message="""When a link is provided, you should ask the assistant for fetching the content. Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
    function_map={"search_google_news": search_google_news}
)

In [ ]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
Search google news in the topic of "Hamas", and write a news based on them.
You should avoid bullet points.
Instead, use more expressive words to form a news that's like one written by a well recognized journalist.

Start the work now.
"""
)

user_proxy (to assistant):


Search google news in the topic of "Hamas", and write a news based on them.
You should avoid bullet points. 
Instead, use more expressive words to form a news that's like one written by a well recognized journalist.

Start the work now.


--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function Call: search_google_news *****
Arguments: 

{
  "keyword": "Hamas"
}
*******************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION search_google_news...
Keyword: Hamas
user_proxy (to assistant):

***** Response from calling function "search_google_news" *****
['https://www.wsj.com/livecoverage/israel-hamas-gaza-rockets-attack-palestinians', 'https://apnews.com/article/israel-palestinians-gaza-hamas-rockets-airstrikes-tel-aviv-11fb98655c256d54ecb5329284fc37d2', 'https://www.bbc.com/news/wo